In [1]:
%matplotlib inline
from __future__ import division

import pandas as pd
import utils
import datetime

In [2]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [5]:
# grab unknowns
unk = pd.read_csv('../data/unqualified_traffic_summary.csv', parse_dates=['date'])
unk = unk.dropna(subset=['domain'])

# since we were only given these at the domain level, will be sufficient to strip urls after .com
unk['stripped_url'] = unk.domain.apply(lambda x: utils.clean_url(x, strip=True))
unk['dt'] = unk.date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

unk = unk.rename(columns={'sessions':'unk_sessions'})
unkd = unk.groupby(['stripped_url', 'dt']).unk_sessions.sum().reset_index()

In [6]:
# strip out disease categories + diseases
disease_categories = pd.DataFrame()
for row in url_categories.fillna('').iterrows():
    d = {}
    r = row[1]
    d['url_category_id'] = r.url_category_id
    dc = r.disease_category
    for x in dc.split(','):
        dis = x.lower().strip()
        if dis != '':
            d['disease_category'] = dis
            disease_categories = disease_categories.append(d, ignore_index=True)
            
diseases = pd.DataFrame()
for row in url_categories.fillna('').iterrows():
    d = {}
    r = row[1]
    d['url_category_id'] = r.url_category_id
    dc = r.disease
    for x in dc.split(','):
        dis = x.lower().strip()
        if dis != '':
            d['disease'] = dis
            diseases = diseases.append(d, ignore_index=True)

In [7]:
# bring in visitors, determine if authorized
categorized_visits = pd.merge(visitors, events, on='dg_id')
categorized_visits['authorized'] = categorized_visits.identity_type.apply(lambda x: 1 if x=='AUT' else 0)

# isolate pharma companies
pharma = categorized_visits[categorized_visits.site_category=='Pharma']
pharma['dt'] = pharma.timestamp.apply(lambda x: x.date())
pharma.dt = pd.to_datetime(pharma.dt)

pharma_activity = pharma.groupby(['category_url', 'dt']).authorized.agg(['sum', 'count']).reset_index()
pharma_activity.columns = ['url', 'dt', 'authorized_visits', 'identified_visits']
pharma_activity = pd.merge(url_categories, pharma_activity, on='url').fillna('')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
# merge with unknowns by date
pharma_merged = pd.merge(pharma_activity, unkd, on=['stripped_url', 'dt'], how='left')
pharma_merged['month'] = pharma_merged.dt.apply(lambda x: x.month)
pharma_merged['total_visits'] = pharma_merged.apply(lambda x: x.identified_visits + x.unk_sessions, axis=1)
pharma_merged['pct_authorized'] = pharma_merged.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [10]:
# keep useful columns
pharma_merged = \
    pharma_merged[['url_category_id', 'dt', 'url', 'site_sub_category', 
               'month', 'disease_category', 'disease', 'pharma_firm', 
               'authorized_visits',  'identified_visits', 'unk_sessions', 
               'total_visits', 'dt_first_view', 'pct_authorized']]

In [23]:
pharma.head()

,visitor_id,dg_id,identity_type,professional_designation,npi_number,primary_specialty,primary_specialty_group,secondary_specialty,secondary_specialty_group,first_name,...,fragment,reason,category_url,site_category,site_sub_category,disease_category,disease,pharma_firm,authorized,dt
107,4412,R0000002268,POI,Nurse,NaN,NaN,NaN,NaN,NaN,ZINA,...,NaN,NaN,www.belsomra.com,Pharma,Branded Consumer,Psychiatry,Insomnia,Merck,0,2017-02-04
222,6259,R0000006692,POI,Nurse,NaN,NaN,NaN,NaN,NaN,YUNNFANG,...,NaN,NaN,www.tarceva.com,Pharma,Branded Consumer,Oncology,"Lung Cancer, pancreatic cancer",Genentech/Astellas,0,2017-07-23
223,6259,R0000006692,POI,Nurse,NaN,NaN,NaN,NaN,NaN,YUNNFANG,...,NaN,NaN,www.tarceva.com,Pharma,Branded Consumer,Oncology,"Lung Cancer, pancreatic cancer",Genentech/Astellas,0,2017-07-23
252,6857,R0000008040,POI,Nurse,NaN,NaN,NaN,NaN,NaN,PROSPECTIVE STUDENT,...,NaN,NaN,www.keytruda.com,Pharma,Branded Consumer,Oncology,"melanoma, lung cancer, head and neck tumors, l...",Merck,0,2017-07-25
258,7899,R0000010415,POI,Nurse,NaN,NaN,NaN,NaN,NaN,YELENA,...,NaN,NaN,www.gardasil9.com,Pharma,Branded Consumer,Pediatrics,Human Papilloma virus,Merck,0,2017-07-05


In [15]:
merck = merck_visits.groupby('url')[['authorized_visits', 'total_visits']].sum().reset_index()
merck['pct_authorized'] = merck.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [19]:
merck.to_csv('../data/merck_authorized_visits.csv', index=False)

In [8]:
pharma_auth = pharma_merged.groupby('pharma_firm')[['authorized_visits', 'total_visits']].sum().reset_index()
pharma_auth['pct_authorized'] = pharma_auth.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [13]:
pharma_auth.to_csv('../data/all_pharma_firms_authorized_visits.csv', index=False)

In [11]:
# we probably only care about monthly granularity
authorized_by_month = pharma_merged.groupby(['url_category_id', 'month'])[['authorized_visits', 
                                                     'identified_visits', 
                                                     'unk_sessions', 
                                                     'total_visits']].sum().reset_index()

In [33]:
pm_visits = authorized_by_month[~authorized_by_month.total_visits.isnull()]
pm_visits['pct_authorized'] = pm_visits.apply(lambda x: x.identified_visits/x.total_visits, axis=1)

print 'mean: ', pm_visits.pct_authorized.mean()
print 'total: ', sum(pm_visits.identified_visits)/sum(pm_visits.total_visits)

mean:  0.0293383455933
total:  0.00436732201266


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
def authorized_visits_by_month(pharma_site):
    disc = pd.DataFrame()
    diss = pd.DataFrame()
    
    ucid = url_categories[url_categories.url==pharma_site].url_category_id.item()
    site_authorized_by_month = authorized_by_month[authorized_by_month.url_category_id==ucid]
    site_authorized_by_month['pct_authorized'] = site_authorized_by_month.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)
    dis_cats = disease_categories[disease_categories.url_category_id==ucid]
    for d in dis_cats.disease_category:
        dc = disease_categories[disease_categories.disease_category==d]
        if len(dc) > 3:
            m = pd.merge(authorized_by_month, dc, on='url_category_id')
            m = m.groupby(['month', 'disease_category'])[['authorized_visits', 'total_visits']].sum().reset_index()
            m['pct_authorized'] = m.authorized_visits/m.total_visits
            m['num_sites'] = len(dc)
            disc = disc.append(m, ignore_index=True)
    dis = diseases[diseases.url_category_id==ucid]
    for d in dis.disease:
        ds = diseases[diseases.disease==d]
        if len(dc) > 3:
            m = pd.merge(authorized_by_month, ds, on='url_category_id')
            m = m.groupby(['month', 'disease'])[['authorized_visits', 'total_visits']].sum().reset_index()
            m['pct_authorized'] = m.authorized_visits/m.total_visits
            m['num_sites'] = len(dc)
            diss = diss.append(m, ignore_index=True)
    
    # merge for tableau
    good_cols = ['url_category_id', 'pharma_site', 'pharma_firm', 'month', 'authorized_visits', 'pct_authorized']
    df = site_authorized_by_month[['url_category_id', 'month', 'authorized_visits', 'pct_authorized']]
    if len(disc)>0:
        disc['url_category_id'] = ucid
        df2 = disc[['url_category_id', 'month', 'disease_category', 'authorized_visits', 'pct_authorized', 'num_sites']]
        df = pd.concat([df, df2])
        good_cols += ['disease_category', 'num_sites']
    if len(diss)>0:
        diss['url_category_id'] = ucid
        df3 = diss[['url_category_id', 'month', 'disease', 'authorized_visits', 'pct_authorized', 'num_sites']]
        df = pd.concat([df, df3])
        good_cols += ['disease']
        
    df['pharma_site'] = pharma_site
    df['pharma_firm'] = url_categories[url_categories.url_category_id==ucid].pharma_firm.item()
    return df[good_cols]

In [10]:
df =  authorized_visits_by_month('www.dulera.com')

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
df.head()

,url_category_id,pharma_site,pharma_firm,month,authorized_visits,pct_authorized,disease_category,num_sites,disease
0,46,www.dulera.com,Merck,2,442,0.004690,NaN,NaN,NaN
1,46,www.dulera.com,Merck,3,683,0.013057,NaN,NaN,NaN
2,46,www.dulera.com,Merck,4,959,0.014179,NaN,NaN,NaN
3,46,www.dulera.com,Merck,5,601,0.017305,NaN,NaN,NaN
4,46,www.dulera.com,Merck,6,543,0.026376,NaN,NaN,NaN


In [20]:
all_urls = pd.DataFrame()
for url in pharma_activity.url.unique():
    all_urls = all_urls.append(authorized_visits_by_month(url))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
all_urls[['url_category_id', 'pharma_site', 'pharma_firm', 'month', 
          'disease_category', 'disease', 'pct_authorized',
          'num_sites', 'authorized_visits']].to_csv('../data/authorized_visits_by_month.csv', index=False)

In [21]:
all_urls.head()

,authorized_visits,disease,disease_category,month,num_sites,pct_authorized,pharma_firm,pharma_site,url_category_id
0,442,NaN,NaN,2,NaN,0.004690,Merck,www.dulera.com,46
1,683,NaN,NaN,3,NaN,0.013057,Merck,www.dulera.com,46
2,959,NaN,NaN,4,NaN,0.014179,Merck,www.dulera.com,46
3,601,NaN,NaN,5,NaN,0.017305,Merck,www.dulera.com,46
4,543,NaN,NaN,6,NaN,0.026376,Merck,www.dulera.com,46


In [13]:
npi_totals = pd.read_excel('../data/AIM Weekly Report2017-08-22_Specialties_Breakout.xlsx')

In [15]:
npi_totals.columns = npi_totals.loc[0]

In [19]:
npi_totals

,Practitioner Type,Type,Specialty,New AIM Tagged Individuals,Total AIM Tagged Individuals,DMD Total Universe,% Coverage
0,Practitioner Type,Type,Specialty,New AIM Tagged Individuals,Total AIM Tagged Individuals,DMD Total Universe,% Coverage
1,Physician,Unknown,Unknown,118,25295,0,-
2,Physician,Unknown,Total,118,25295,0,-
3,Physician,Internal Medicine,Internal Medicine (IM),292,61686,144556,0.426727
4,Physician,Internal Medicine,Gastroenterology (GE),36,8898,16252,0.547502
5,Physician,Internal Medicine,Nephrology (NEP),37,6267,11444,0.547623
6,Physician,Internal Medicine,Pulmonary Critical Care Medicine (PCC),51,5282,9243,0.571459
7,Physician,Internal Medicine,Infectious Disease (ID),32,4820,9373,0.514243
8,Physician,Internal Medicine,Pulmonary Disease (PUD),17,3332,5595,0.595532
9,Physician,Internal Medicine,Rheumatology (RHU),27,3306,6274,0.526937
